In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

import re
import json
import io

In [2]:
url = "https://www.thebalance.com/trump-s-tax-plan-how-it-affects-you-4113968"

In [3]:
resp = requests.get(url)

In [4]:
resp.text

'<!DOCTYPE html>\n\t\n\n\n\n\n\t\t<html id="flexTemplate_1-0" class="comp flexTemplate html mntl-layout-html taxlevel-3" data-ab="20,25,23,1,4,99,99,99" data-resource-version="2.78.1" lang="en" data-money-resource-version="2.78.1" data-mantle-resource-version="3.5.69">\n<!--\n<globe-environment environment="prod" application="money" dataCenter="us-east-1" serverName="ip-10-14-9-92-1" />\n<globe-server version="2.78.1" vendor="" title="money-server" />\n<globe-resources version="2.78.1"  />\n-->\n\t\t<head class="loc head">\n<script type="text/javascript"></script>    <link rel="preconnect" href="//www.googletagmanager.com">\n    <link rel="preconnect" href="//www.googletagservices.com">\n    <link rel="preconnect" href="//www.google-analytics.com">\n    <link rel="preconnect" href="//tpc.googlesyndication.com">\n    <link rel="preconnect" href="//pagead2.googlesyndication.com">\n    <link rel="preconnect" href="//cdn.doubleverify.com">\n    <link rel="preconnect" href="//cdn3.doublever

In [5]:
with open('saved_page.html', 'w') as f:
    f.write(resp.text)

In [6]:
soup = BeautifulSoup(resp.text,'lxml')

In [7]:
table = soup.find_all('tbody')[0]
data = {}
names = {}
cleanse = r'[^a-zA-Z0-9\-\.]+'
for i,row in enumerate(table.find_all('tr')):
    inner_list = []
    if i == 0:
        for k, col in enumerate(row.find_all('th')):
            clean = re.sub(cleanse,'',col.text)
            names[k] = clean
            data[names[k]] = []
    else:
        for k,col in enumerate(row.find_all('td')):
            clean = re.sub(cleanse,'',col.text)
            data[names[k]].append(clean)
            
tax_rates = pd.DataFrame(data)
tax_rates

,2017,2018-2025,Married-Joint,Single
0,10,10,0-19050,0-9525
1,15,12,19050-77400,9525-38700
2,25,22,77400-165000,38700-82500
3,28,24,165000-315000,82500-157500
4,33,32,315000-400000,157500-200000
5,33-35,35,400000-600000,200000-500000
6,39.6,37,600000,500000


In [8]:
tax_rates.loc[5,'2017'] = 34

In [9]:
tax_rates.loc[6, 'Married-Joint'] = '600000-600000'
tax_rates.loc[6, 'Single'] = '500000-500000'

In [10]:
rate_2017 = tax_rates.copy()
rate_2018 = tax_rates.copy()

In [11]:
rate_2017['year'] = 2017
rate_2018['year'] = 2018

In [12]:
rate_2018['rate'] = rate_2018['2018-2025'].apply(lambda x:float(x)/100)
rate_2017['rate'] = rate_2017['2017'].apply(lambda x:float(x)/100)

In [13]:
filing_single_2017 = rate_2017.copy()
filing_joint_2017 = rate_2017.copy()
filing_single_2018 = rate_2018.copy()
filing_joint_2018 = rate_2018.copy()

In [14]:
filing_joint_2017['min_range'] = filing_joint_2017['Married-Joint'].apply(lambda x:float(x.split('-')[0]))
filing_joint_2017['max_range'] = filing_joint_2017['Married-Joint'].apply(lambda x:float(x.split('-')[1]))
filing_joint_2017['bracket_order'] = filing_joint_2017.index
filing_joint_2017['filing_type'] = 'joint'
filing_single_2017['min_range'] = filing_single_2017['Single'].apply(lambda x:float(x.split('-')[0]))
filing_single_2017['max_range'] = filing_single_2017['Single'].apply(lambda x:float(x.split('-')[1]))
filing_single_2017['bracket_order'] = filing_single_2017.index
filing_single_2017['filing_type'] = 'single'
filing_joint_2018['min_range'] = filing_joint_2018['Married-Joint'].apply(lambda x:float(x.split('-')[0]))
filing_joint_2018['max_range'] = filing_joint_2018['Married-Joint'].apply(lambda x:float(x.split('-')[1]))
filing_joint_2018['bracket_order'] = filing_joint_2018.index
filing_joint_2018['filing_type'] = 'joint'
filing_single_2018['min_range'] = filing_single_2018['Single'].apply(lambda x:float(x.split('-')[0]))
filing_single_2018['max_range'] = filing_single_2018['Single'].apply(lambda x:float(x.split('-')[1]))
filing_single_2018['bracket_order'] = filing_single_2018.index
filing_single_2018['filing_type'] = 'single'

In [15]:
keep_fields = ['bracket_order','filing_type','year','rate','min_range','max_range']

In [16]:
parsed_lookup_table = pd.concat(
    [
        filing_joint_2017, 
        filing_single_2017, 
        filing_joint_2018, 
        filing_single_2018,
    ]
)

In [18]:
tax_brackets['taxable_income'] = tax_brackets['max_range'] - tax_brackets['min_range']

In [19]:
tax_brackets.dtypes

bracket_order       int64
filing_type        object
year                int64
rate              float64
min_range         float64
max_range         float64
taxable_income    float64
dtype: object

In [20]:
tax_brackets.to_csv('parsed_tax_brackets.csv',index=None)

In [21]:
%%bash
open parsed_tax_brackets.csv

In [ ]:
tax_brackets